In [1]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

2.5.0


In [3]:
!wget https://raw.githubusercontent.com/davidsbatista/NER-datasets/master/CONLL2003/test.txt

--2021-05-26 12:00:05--  https://raw.githubusercontent.com/davidsbatista/NER-datasets/master/CONLL2003/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748093 (731K) [text/plain]
Saving to: ‘test.txt’

test.txt            100%[===================>] 730.56K  --.-KB/s    in 0.1s    

2021-05-26 12:00:06 (6.06 MB/s) - ‘test.txt’ saved [748093/748093]



In [4]:
!wget https://raw.githubusercontent.com/davidsbatista/NER-datasets/master/CONLL2003/train.txt

--2021-05-26 12:00:11--  https://raw.githubusercontent.com/davidsbatista/NER-datasets/master/CONLL2003/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283418 (3.1M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   3.13M  10.4MB/s    in 0.3s    

2021-05-26 12:00:12 (10.4 MB/s) - ‘train.txt’ saved [3283418/3283418]



In [5]:
!wget https://raw.githubusercontent.com/davidsbatista/NER-datasets/master/CONLL2003/valid.txt

--2021-05-26 12:00:16--  https://raw.githubusercontent.com/davidsbatista/NER-datasets/master/CONLL2003/valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 827441 (808K) [text/plain]
Saving to: ‘valid.txt’

valid.txt           100%[===================>] 808.05K  --.-KB/s    in 0.1s    

2021-05-26 12:00:17 (6.26 MB/s) - ‘valid.txt’ saved [827441/827441]



In [6]:
labels = set()

def file2Examples(file_name):
  '''
  Read data files and return input/output pairs
  '''
  
  examples=[]

  with open(file_name,"r") as f:

    next(f)
    next(f)

    example = [[],[]]

    for line in f:

      input_output_split= line.split()

      if len(input_output_split)==4:
        example[0].append(input_output_split[0])
        example[1].append(input_output_split[-1])
        labels.add(input_output_split[-1])

      elif len(input_output_split)==0:
        examples.append(example)
        example=[[],[]]
      else:
        example=[[],[]]

    f.close()
    
    return examples
  
# Extract examples from train, validation, and test files which can be found at 
# https://github.com/davidsbatista/NER-datasets/tree/master/CONLL2003
train_examples = file2Examples("train.txt")
test_examples = file2Examples("test.txt")
valid_examples = file2Examples("valid.txt")

In [7]:
!head train.txt

-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O


In [8]:
    # create character vocab
    all_text = " ".join([" ".join(x[0]) for x in train_examples+valid_examples+test_examples])
    vocab = sorted(set(all_text))
    
    # create character/id and label/id mapping
    char2idx = {u:i+1 for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    label2idx = {u:i+1 for i, u in enumerate(labels)}
    idx2label = np.array(labels)
    
    print(idx2label)
    print(char2idx)

{'B-ORG', 'I-ORG', 'I-MISC', 'B-MISC', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-LOC'}
{' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '=': 29, '?': 30, '@': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, ']': 59, '`': 60, 'a': 61, 'b': 62, 'c': 63, 'd': 64, 'e': 65, 'f': 66, 'g': 67, 'h': 68, 'i': 69, 'j': 70, 'k': 71, 'l': 72, 'm': 73, 'n': 74, 'o': 75, 'p': 76, 'q': 77, 'r': 78, 's': 79, 't': 80, 'u': 81, 'v': 82, 'w': 83, 'x': 84, 'y': 85, 'z': 86}


In [9]:
    def split_char_labels(eg):
      '''
      For a given input/output example, break tokens into characters while keeping 
      the same label.
      '''

      tokens = eg[0]
      labels=eg[1]

      input_chars = []
      output_char_labels = []

      for token,label in zip(tokens,labels):

        input_chars.extend([char for char in token])
        input_chars.extend(' ')
        output_char_labels.extend([label]*len(token))
        output_char_labels.extend('O')

      return [[char2idx[x] for x in input_chars[:-1]],np.array([label2idx[x] for x in output_char_labels[:-1]])]
   
    train_formatted = [split_char_labels(eg) for eg in train_examples]
    test_formatted = [split_char_labels(eg) for eg in test_examples]
    valid_formatted = [split_char_labels(eg) for eg in valid_examples]
    
    print(len(train_formatted))
    print(len(test_formatted))
    print(len(valid_formatted))

14985
3682
3464


In [11]:
    # training generator
    def gen_train_series():

        for eg in train_formatted:
          yield eg[0],eg[1]
    
    # validation generator
    def gen_valid_series():
    
       for eg in valid_formatted:
          yield eg[0],eg[1]
    
    # test generator
    def gen_test_series():

      for eg in test_formatted:
          yield eg[0],eg[1]
      
    # create Dataset objects for train, test and validation sets  
    series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
    series_valid = tf.data.Dataset.from_generator(gen_valid_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
    series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

    BATCH_SIZE = 128
    BUFFER_SIZE=1000
    
    # create padded batch series objects for train, test and validation sets
    ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    ds_series_batch_valid = series_valid.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    
    # print example batches
    for input_example_batch, target_example_batch in ds_series_batch_valid.take(1):
      print(input_example_batch)
      print(target_example_batch)


tf.Tensor(
[[34 49 40 ...  0  0  0]
 [43 46 45 ...  0  0  0]
 [54 65 79 ...  0  0  0]
 ...
 [ 3  1 36 ...  0  0  0]
 [40 66  1 ...  0  0  0]
 [35 81 78 ...  0  0  0]], shape=(128, 228), dtype=int32)
tf.Tensor(
[[6 6 6 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 ...
 [6 6 6 ... 0 0 0]
 [6 6 6 ... 0 0 0]
 [7 7 7 ... 0 0 0]], shape=(128, 228), dtype=int32)


In [12]:
  vocab_size = len(vocab)+1

  # The embedding dimension
  embedding_dim = 256

  # Number of RNN units
  rnn_units = 1024

  label_size = len(labels)  
  
  # build LSTM model
  def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None],mask_zero=True),
            tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(label_size)
            ])
        return model

  model = build_model(
        vocab_size = len(vocab)+1,
        label_size=len(labels)+1,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

  model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          22272     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 10)           10250     
Total params: 5,279,498
Trainable params: 5,279,498
Non-trainable params: 0
_________________________________________________________________


In [13]:
    import os

    # define loss function
    def loss(labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

    model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    # Directory where the checkpoints will be saved
    checkpoint_dir = './training_checkpoints'
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

In [17]:
    EPOCHS=20
    history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_valid,callbacks=[checkpoint_callback])

Epoch 1/20
117/117 [==============================] - 17s 146ms/step - loss: 0.0931 - sparse_categorical_accuracy: 0.8753 - val_loss: 0.0963 - val_sparse_categorical_accuracy: 0.8778
Epoch 2/20
117/117 [==============================] - 17s 146ms/step - loss: 0.0849 - sparse_categorical_accuracy: 0.8865 - val_loss: 0.0882 - val_sparse_categorical_accuracy: 0.8889
Epoch 3/20
117/117 [==============================] - 17s 147ms/step - loss: 0.0799 - sparse_categorical_accuracy: 0.8938 - val_loss: 0.0835 - val_sparse_categorical_accuracy: 0.8967
Epoch 4/20
117/117 [==============================] - 17s 146ms/step - loss: 0.0765 - sparse_categorical_accuracy: 0.8992 - val_loss: 0.0805 - val_sparse_categorical_accuracy: 0.8997
Epoch 5/20
117/117 [==============================] - 17s 146ms/step - loss: 0.0722 - sparse_categorical_accuracy: 0.9042 - val_loss: 0.0788 - val_sparse_categorical_accuracy: 0.9022
Epoch 6/20
117/117 [==============================] - 17s 147ms/step - loss: 0.0691 -

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:

  pred=model.predict_on_batch(input_example_batch)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  y_true=target_example_batch.numpy().flatten()

  preds=np.concatenate([preds,pred_max])
  y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))

[[  4212     79     29    602    826    584    579     15      8]
 [    64   3120    113     26     69    646     89    517    198]
 [    27    382    461     27     10    340     24    206    129]
 [   897     68     41   2413    984    526    371     41     13]
 [  2006     51      0    788   7100    727    738     13      8]
 [  1809    400    223    661    774 186850    937    184    124]
 [  1091     66      8    299    664    655   6285     42     14]
 [    53    318     64     17     22    120     24   6049     58]
 [     3    249     71      4      0     72     16    210    820]]
              precision    recall  f1-score   support

         1.0       0.41      0.61      0.49      6934
         2.0       0.66      0.64      0.65      4842
         3.0       0.46      0.29      0.35      1606
         4.0       0.50      0.45      0.47      5354
         5.0       0.68      0.62      0.65     11431
         6.0       0.98      0.97      0.98    191962
         7.0       0.69   